In [ ]:
import yaml
import numpy as np
import pandas as pd
import lhapdf as lh
import matplotlib.pyplot as plt
import MatplotlibSettings
%matplotlib inline
from collections import deque
import copy
import os
import re
from os import listdir
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# Fit folder
FitFolder = "../"

In [ ]:
# Read and report chi2's
with open(FitFolder + "/Chi2s.yaml", "r") as chi2file:
    for chi2 in yaml.load(chi2file, Loader = yaml.CLoader):
        for c in chi2.items(): 
            print(c[0],"(Npt = ",c[1]["Npt"],"): chi2 = ", c[1]["chi2"])

In [ ]:
# Map config's observables to y-labels
obs_map = {"g1":r"$\displaystyle{g_1}$",
           "g1_F1":r"$\displaystyle\frac{g_1}{F_1}$",
           "A1H":r"$\displaystyle{A_1^H}$",
           "a3":r"$\displaystyle{a_3}$",
           "a8":r"$\displaystyle{a_8}$"}

# Open config file
config = yaml.load(open(FitFolder + "/config.yaml", "r"), Loader = yaml.CLoader)

exps = [names["name"] for names in config.get("Data")["sets"]] 
ylabels = dict()
plotfile = dict()
xlims = dict()
xlog = dict()
binints = dict()
ylims1 = dict()


for dataset in config.get("Data")["sets"]:
    
    with open(FitFolder + "/data/" + dataset['file'], "r") as FileName:
        data = yaml.load(FileName, Loader = yaml.CLoader)
        plotfile[dataset['name']] = dataset['file'].replace(".yaml", "")
        xlog[dataset['name']] = True
        
        for ql in data['dependent_variables'][0]['qualifiers']:
            if ql['name'] == "observable":
                ylabels[dataset['name']] = obs_map[ql['value']]
            elif ql['name'] == "x":
                xlims[dataset['name']] = [ql['low'], ql['high']]
                binints[dataset['name']] = ql['integrate']

binints['a3 sum rule'] = False
binints['a8 sum rule'] = False
                
#Create plots folder
if not os.path.exists(FitFolder + "/plots"):
    os.makedirs(FitFolder + "/plots")

## $\chi^2$ profiles

In [ ]:
# Chi2 profiles
for frep in [f.replace(".yaml", "") for f in sorted(listdir(FitFolder + "/log/"),  key=lambda s: int(s[s.find("_")+1:s.find(".")]))]:
    with open(FitFolder + "/log/" + frep + ".yaml", "r") as rep:
        replica = yaml.load(rep, Loader = yaml.CLoader)
        ite   = [r["iteration"]       for r in replica]
        chi2t = [r["training chi2"]   for r in replica]
        chi2v = [r["validation chi2"] for r in replica]
        
        plt.title(r"\textbf{" + frep.replace("_", " ") + "}")
        plt.ylabel(r"\textbf{Error function}")
        plt.xlabel(r"\textbf{Iteration}")
        plt.xscale("log")
        plt.yscale("log")
        plt.ylim(1, 10)
        plt.xlim(10, 3000)
        plt.plot(ite, chi2t, color = "blue", label = r"\textbf{Training}")
        plt.plot(ite, chi2v, color = "red",  label = r"\textbf{Validation}")
        plt.legend()
        #plt.savefig(FitFolder + "/" + frep + ".pdf")
        plt.show()
        plt.close()

## SIDIS + DIS: predictions Vs. experimental points

In [ ]:
# SIDIS + DIS
with open(FitFolder + "/Predictions.yaml", "r") as p:
    preds = yaml.load(p, Loader = yaml.CLoader)
    for e in preds:
        for exp, points in e.items():
            #if ("a3 sum rule" in exp) or ("a8 sum rule" in exp):
             #   continue
            print(exp)
            xav  = np.array([pt["xav"] for pt in points])
            excv = np.array([pt["exp. central value"] for pt in points])
            exun = np.array([pt["exp. unc."] for pt in points])
            thcv = np.array([pt["prediction"] for pt in points])
            thun = np.array([pt["pred. unc."] for pt in points])

            f, (ax1, ax2) = plt.subplots(2, 1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [4, 1]))
            plt.subplots_adjust(wspace = 0, hspace = 0)

            # Upper panel
            ax1.set_xlim(auto=True)
            ax1.set_ylim(auto=True)
            ax1.set_ylabel(ylabels[exp])
            ax1.set_yscale("log")
            if xlog[exp]:
                ax1.set_xscale("log")
            ax1.errorbar(xav, excv, exun, elinewidth = 2, capsize = 3, capthick = 1.5, markersize = 5, fmt = "ko")
            if binints[exp]:
                ax1.bar(xmin, 2 * thun, bottom = thcv - thun, width = xmax - xmin, align = "edge", color = "red", label = r"\textbf{NNLO + MAP FF1.0}", alpha = 0.5)
            else:
                ax1.errorbar(xav, thcv, thun, elinewidth = 2, capsize = 3, capthick = 1.5, label = r"\textbf{NNLO + MAP FF1.0}", markersize = 4, fmt = "rs")
            ax1.legend(fontsize = 20)

            # Lower panel
            ax2.set_xlim(xlims[exp])
            ax2.set_ylim(auto=True)
            ax2.set_xlabel(r"$x$")
            ax2.set_ylabel(r"\textbf{Ratio to Data}", fontsize = 16)
            if xlog[exp]:
                ax2.set_xscale("log")
            ax2.axhline(y = 1, c = "k", ls = "--", lw = 1.5)
            
            ax2.errorbar(xav, excv/excv, exun/abs(excv), elinewidth = 2, capsize = 3, capthick = 1.5, markersize = 5, fmt = "ko")
            if binints[exp]:
                ax2.bar(xmin, 2 * thun/abs(excv), bottom = (thcv - thun)/abs(excv), width = xmax - xmin, align = "edge", color = "red", alpha = 0.5)
            else:
                ax2.errorbar(xav, thcv/abs(excv), thun/abs(excv), elinewidth = 2, capsize = 3, capthick = 1.5, markersize = 5, fmt = "rs")

            #plt.savefig(FitFolder + "/plots/" + plotfile[exp] + ".pdf")
            plt.show()
            plt.close()

## Polarised PDF plots

In [ ]:
# Upload sets and declare uncertainty type
lh.pathsAppend(str(FitFolder))
pdfs = [lh.mkPDFs("LHAPDFSet"), lh.mkPDFs("NNPDFpol11_100")]
unc = ["montecarlo", "montecarlo"]
colff = ["red", "blue"]
nameff = [r"\textbf{MAPPDFpol1.0}", r"\textbf{NNPDFpol1.1}"]

# Upload unpolarised PDF set
unp_pdf = [lh.mkPDFs("NNPDF31_nnlo_pch_as_0118")]
unp_unc = ["montecarlo"]
unp_color = "green"
name_unp_pdf = r"\textbf{NNPDF3.1 nnlo}"

# Function that returns the central values
def ComputeCentralValueAndUncertainty(x, Q, comb, pdfsets, unc_type):
    centralvalue = []
    uncertainty  = []
    for s in zip(pdfsets, unc_type):
        if s[1] == "hessian":
            # In the case of a hessian set, use replica 0 as central value
            cv = 0
            f = s[0][0].xfxQ(x, Q)
            for k, v in comb.items():
                cv += v * f[k]
            err = 0
            for im in range(int((len(s[0]) - 1)/2)):
                fp = s[0][2*im+1].xfxQ(x, Q)
                fm = s[0][2*im+2].xfxQ(x, Q)
                t = 0
                for k, v in comb.items():
                    t += v * fp[k]
                    t -= v * fm[k]
                err += t**2
            centralvalue.append(cv)
            uncertainty.append(np.sqrt(err)/2)            
        elif s[1] == "montecarlo":
            # In case of a MC set, compute the average and standard deviation (do not include replica 0)
            cv  = 0
            cv2 = 0
            for irep in range(1, len(s[0])):
                f = s[0][irep].xfxQ(x, Q)
                t = 0
                for k, v in comb.items():
                    t += v * f[k]
                cv  += t / ( len(s[0]) - 1 )
                cv2 += t**2 / ( len(s[0]) - 1 )
            centralvalue.append(cv)
            uncertainty.append(np.sqrt(cv2 - cv**2))
        else:
            sys.exit("Unknown error: ", s[1])
    return centralvalue, uncertainty

# Function that computes polPDF in the chosen basis
def ComputeCombintaion(x, comb, func):
    temp = 0
    for k, v in comb.items():
        temp += v*func.xfxQ(k,x,Q)
    return temp

### Polarised PDF plot settings

In [ ]:
# Scale
Q = 1

# Grid in x
xv = np.logspace(-5, -0.0001, 1000)

# Combinations to be plotted
# ----------------- Evolution basis --------------------
combs_ev = [{21: 1},                                    # Delta_G
            {1: 1, -1: 1, 2: 1, -2: 1, 3: 1, -3: 1},    # Delta_Sigma
            {1: 1, -1: -1, 2: 1, -2: -1},               # Delta_V
            {2: 1, -2: 1, 1: -1, -1: -1},               # Delta_T3
            {2: 1, -2: -1, 1: -1, -1: 1},               # Delta_V3
            {1: 1, -1: 1, 2: 1, -2: 1, 3: -2, -3: -2 }] # Delta_T8 
labels_ev = [r"$x \Delta G(x,Q)$", 
             r"$x \Delta \Sigma(x,Q)$", 
             r"$x \Delta V(x,Q)$", 
             r"$x \Delta T_3(x,Q)$",
             r"$x \Delta V_3(x,Q)$", 
             r"$x \Delta T_8(x,Q)$"]
pdfname_ev = ["delta_G", 
              "delta_Sigma", 
              "delta_V", 
              "delta_T3", 
              "delta_V3", 
              "delta_T8"]
ylim_ev = [[-1, 1], 
           [-1, 1], 
           [-1, 1], 
           [-1, 1],
           [-1, 1], 
           [-1, 1]]

# ----------------- Flavour basis --------------------
combs_flav = [{21: 1},  # Delta_G
              {1: 1},   # Delta_d
              {-1:1},   # Delta_dbar
              {2: 1},   # Delta_u
              {-2:1},   # Delta_ubar
              {3:1},    # Delta_s
              {-3:1}]   # Delta_sbar
labels_flav = [r"$x \Delta g(x,Q)$", 
               r"$x \Delta d(x,Q)$", 
               r"$x \Delta \overline{d}(x,Q)$", 
               r"$x \Delta u(x,Q)$",
               r"$x \Delta \overline{u}(x,Q)$",
               r"$x \Delta s(x,Q)$",
               r"$x \Delta \overline{s}(x,Q)$"]
pdfname_flav = ["Delta_G", 
                "Delta_d", 
                "Delta_dbar", 
                "Delta_u", 
                "Delta_ubar", 
                "Delta_s",
                "Delta_sbar"]
ylim_flav = [[-1, 1], 
             [-0.3, 0.2], 
             [-0.15, 0.3], 
             [-0.25, 0.5], 
             [-0.15, 0.15], 
             [-0.10, 0.05],
             [-0.10, 0.10]]

### Replicas

In [ ]:
show_unp = False

# Run over combinations
for cm in zip(combs_flav, labels_flav, pdfname_flav, ylim_flav) :
    
    # Set up the configuration of the plot
    f, ax1 = plt.subplots(1,1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [3]))

    # Set the labels and the x-range
    ax1.set_title(r"$Q = " + str(Q) + r"$ \textbf{GeV}")
    ax1.set_ylabel(cm[1])
    ax1.set_xlim([0.001, 1])
    ax1.set_ylim(cm[3])
    ax1.plot(xv, xv - xv, color = "black", ls = "--", lw = 1, zorder=5)

    # Plot comparative function    
    ax1.plot(xv, [ComputeCombintaion(x, cm[0], pdfs[1][0]) for x in xv], ls = "-", lw = 2, color = colff[1], label=(nameff[1] + r" \textbf{replica 0}"), zorder=10)

    if 21 in cm[0].keys() and show_unp:

        unp_fv  = np.zeros(shape=(xv.size, 1))
        unp_dfv = np.zeros(shape=(xv.size, 1))

        for i, x in enumerate(xv):
            unp_fv[i], unp_dfv[i] = ComputeCentralValueAndUncertainty(x, Q, combs_flav[0], unp_pdf, unp_unc)

        unp_fv = np.transpose(unp_fv)
        unp_dfv = np.transpose(unp_dfv)
        
        ax1.plot(xv, unp_fv[0], color = "green", ls = "-", lw = 1)
        ax1.fill_between(xv, unp_fv[0] + unp_dfv[0], unp_fv[0] - unp_dfv[0], color = "green", alpha = 0.4, label = name_unp_pdf)
        ax1.plot(xv, -unp_fv[0], color = "green", ls = "-", lw = 1)
        ax1.fill_between(xv, -unp_fv[0] + unp_dfv[0], -unp_fv[0] - unp_dfv[0], color = "green", alpha = 0.4)
    
    # Plot all replicas
    for pdf in pdfs[0]:
        ax1.plot(xv, [ComputeCombintaion(x, cm[0],pdf) for x in xv], ls = "-", lw = 0.2, color = colff[0], zorder=0)
    
    plt.legend()
    #plt.savefig(FitFolder + "plots/" + cm[2])
    plt.show()
    plt.close()

del f, ax1

### Plots polarised PDFs

In [ ]:
# Plot polPDFs
for cm in zip(combs_flav, labels_flav, pdfname_flav, ylim_flav) :
    fv  = np.zeros(shape=(xv.size, len(pdfs)))
    dfv = np.zeros(shape=(xv.size, len(pdfs)))
    
    for i, x in enumerate(xv):
        fv[i], dfv[i] = ComputeCentralValueAndUncertainty(x, Q, cm[0], pdfs, unc)
        
    fv = np.transpose(fv)
    dfv = np.transpose(dfv)

    # Setup plot
    f, (ax1, ax2) = plt.subplots(2, 1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [3, 1]))
    plt.subplots_adjust(wspace = 0, hspace = 0)
    
    ax1.set_title(r"$Q = " + str(Q) + r"$\textbf{GeV} \textbf{SIDIS + DIS + $a_3$ + $a_8$}")
    ax1.set_ylabel(cm[1])
    ax1.set_xlim([0.001, 1])
    ax1.set_xscale("log")
    ax1.set_ylim(cm[3])
    
    ax2.set_xlabel(r"$x$")
    ax2.set_ylabel(r"\textbf{Ratio to}" "\n" r"" + nameff[0], fontsize = 16, labelpad=20.0)
    ax2.set_xlim([0.001, 1])
    ax2.set_xscale("log")
    ax2.set_ylim([0, 2])

    for iset in range(len(fv)):
        ax1.plot(xv, fv[iset], color = colff[iset], ls = "-", lw = 1)
        ax1.fill_between(xv, np.array(fv[iset]) + np.array(dfv[iset]), np.array(fv[iset]) - np.array(dfv[iset]), color = colff[iset], alpha = 0.3, label = nameff[iset])
        ax1.plot(xv, xv - xv, color = "black", ls = "--", lw = 1)
        ax2.plot(xv, np.array(fv[iset])/np.array(fv[0]), color = colff[iset], ls = "-", lw = 1)
        ax2.fill_between(xv, (np.array(fv[iset])+np.array(dfv[iset]))/np.array(fv[0]), (np.array(fv[iset])-np.array(dfv[iset]))/np.array(fv[0]), color = colff[iset], alpha = 0.3)
    ax1.legend()

    #plt.savefig(FitFolder + "plots/" + cm[2] + ".pdf")
    plt.show()
    plt.close()
    
del f, ax1, ax2

### Positivity

In [ ]:
ylim = [[-1, 1], [-0.5, 0.5], [-0.3, 0.3], [-0.5, 0.5], [-0.3, 0.3], [-0.3, 0.3]]

for cm in zip(combs_flav, labels_flav, pdfname_flav, ylim):
    fv  = np.zeros(shape=(xv.size, len(pdfs)))
    dfv = np.zeros(shape=(xv.size, len(pdfs)))
    unp_fv  = np.zeros(shape=(xv.size, 1))
    unp_dfv = np.zeros(shape=(xv.size, 1))

    for i, x in enumerate(xv):
        fv[i], dfv[i] = ComputeCentralValueAndUncertainty(x, Q, cm[0], pdfs, unc)
        unp_fv[i], unp_dfv[i] = ComputeCentralValueAndUncertainty(x, Q, cm[0], unp_pdf, unp_unc)

    fv = np.transpose(fv)
    dfv = np.transpose(dfv)
    unp_fv = np.transpose(unp_fv)
    unp_dfv = np.transpose(unp_dfv)

    # Setup plot
    f, ax1 = plt.subplots(1,1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [3]))
    plt.subplots_adjust(wspace = 0, hspace = 0)

    ax1.set_title(r"$Q = " + str(Q) + r"$ \textbf{GeV} \textbf{SIDIS + DIS + $a_3$ + $a_8$}")
    ax1.set_ylabel(cm[1])
    ax1.set_xlim([0.5, 1])
    ax1.set_xscale("log")
    ax1.set_ylim(cm[3])

    # Plot polarised PDFs
    ax1.plot(xv, fv[0], color = colff[0], ls = "-", lw = 1)
    ax1.fill_between(xv, np.array(fv[0]) + np.array(dfv[0]), np.array(fv[0]) - np.array(dfv[0]), color = colff[0], alpha = 0.4, label = nameff[0])
    ax1.plot(xv, xv - xv, color = "black", ls = "--", lw = 1)

    # Plot unpolarised PDFs
    ax1.plot(xv, unp_fv[0], color = unp_color, ls = "-", lw = 1)
    ax1.fill_between(xv, np.array(unp_fv[0]) + np.array(unp_dfv[0]), np.array(unp_fv[0]) - np.array(unp_dfv[0]), color = unp_color, alpha = 0.4, label = name_unp_pdf)
    ax1.plot(xv, -unp_fv[0], color = unp_color, ls = "-", lw = 1)
    ax1.fill_between(xv, np.array(-unp_fv[0]) + np.array(unp_dfv[0]), np.array(-unp_fv[0]) - np.array(unp_dfv[0]), color = unp_color, alpha = 0.4)
    
    ax1.legend()
    
    #plt.savefig(FitFolder + "/plots/pos_" + cm[2] + ".pdf")
    plt.show()
    plt.close()